In [1]:
!pip install yahoofinancials
import yahoofinancials

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 10

from datetime import date
from datetime import timedelta

from yahoofinancials import YahooFinancials

from scipy.stats import norm

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation

import warnings
warnings.filterwarnings('ignore')

  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15190 sha256=6c0f589f36a85092866b33ee18e1f3174d9b5ab74fbde6cbf64bb620ca307ce8
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [2]:
end = '2021-08-03'
start = '2016-07-20'

stock_symbol = 'TCS.NS'

yf = YahooFinancials(stock_symbol)
data = yf.get_historical_price_data(start, end, 'daily')
TCS = pd.DataFrame(data['TCS.NS']['prices'])

df = TCS.dropna()
df

,date,high,low,open,close,volume,adjclose,formatted_date
0,1468986300,1251.250000,1224.074951,1225.000000,1247.474976,2815252.0,1129.042114,2016-07-20
1,1469072700,1259.750000,1232.224976,1244.750000,1253.025024,2527148.0,1134.065430,2016-07-21
2,1469159100,1262.000000,1246.000000,1249.449951,1257.550049,1517216.0,1138.160645,2016-07-22
3,1469418300,1281.599976,1252.900024,1256.000000,1279.474976,1688796.0,1161.004517,2016-07-25
4,1469504700,1284.000000,1266.500000,1279.474976,1276.300049,2311290.0,1158.123901,2016-07-26
...,...,...,...,...,...,...,...,...
1238,1627357500,3210.000000,3161.149902,3200.000000,3182.949951,1121367.0,3182.949951,2021-07-27
1239,1627443900,3204.449951,3132.399902,3188.000000,3197.800049,1660654.0,3197.800049,2021-07-28
1240,1627530300,3225.449951,3190.300049,3200.250000,3195.800049,1731997.0,3195.800049,2021-07-29
1241,1627616700,3223.500000,3162.350098,3219.000000,3167.449951,2999943.0,3167.449951,2021-07-30


In [3]:
df = df[["formatted_date", "close"]]
df.rename(columns={"formatted_date": "ds", "close": "y"}, inplace=True)
df

,ds,y
0,2016-07-20,1247.474976
1,2016-07-21,1253.025024
2,2016-07-22,1257.550049
3,2016-07-25,1279.474976
4,2016-07-26,1276.300049
...,...,...
1238,2021-07-27,3182.949951
1239,2021-07-28,3197.800049
1240,2021-07-29,3195.800049
1241,2021-07-30,3167.449951


In [8]:
cut_date = '2021-06-01'
mask1 = (df['ds'] <= cut_date)
mask2 = (df['ds'] > cut_date)

X_tr = df.loc[mask1]
X_tst = df.loc[mask2]
print("train shape",X_tr.shape)
print("test shape",X_tst.shape)

train shape (1198, 2)
test shape (43, 2)


In [4]:
!pip install neuralprophet

     |████████████████████████████████| 56 kB 2.2 MB/s 
     |████████████████████████████████| 748.8 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.6.0 which is incompatible.


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from neuralprophet import NeuralProphet


In [12]:
m = NeuralProphet()
#metrics = m.fit(df, validate_each_epoch=True, valid_p=0.2)
model = m.fit(X_tr, validate_each_epoch=True, freq="B", epochs=100) # epoch=1000


INFO: nprophet.utils - set_auto_seasonalities: Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO: nprophet.config - set_auto_batch_epoch: Auto-set batch_size to 8


  0%|          | 0/100 [00:00<?, ?it/s]

INFO: nprophet - _lr_range_test: learning rate range test found optimal lr: 3.51E-01
Epoch[100/100]: 100%|██████████| 100/100 [00:25<00:00,  4.00it/s, SmoothL1Loss=0.0018, MAE=64.1, RegLoss=0, MAE_val=747, SmoothL1Loss_val=0.19]


In [18]:
future = m.make_future_dataframe(X_tr, periods=43)
forecast = m.predict(future)
forecast.head()

,ds,y,yhat1,residual1,trend,season_yearly,season_weekly
0,2021-06-02,None,2014.441650,NaN,1919.543823,35.478485,59.419403
1,2021-06-03,None,2018.103394,NaN,1919.383545,37.486755,61.233120
2,2021-06-04,None,2014.777954,NaN,1919.223267,39.500370,56.054363
3,2021-06-07,None,2021.593628,NaN,1918.742432,45.500019,57.351151
4,2021-06-08,None,2026.236450,NaN,1918.582153,47.459816,60.194504


In [16]:
forecast.tail()

,ds,y,yhat1,residual1,trend,season_yearly,season_weekly
39,2021-07-27,None,2055.794434,NaN,1910.728760,84.871269,60.194504
40,2021-07-28,None,2056.528320,NaN,1910.568604,86.540520,59.419403
41,2021-07-29,None,2059.921143,NaN,1910.408203,88.279778,61.233120
42,2021-07-30,None,2056.380615,NaN,1910.247925,90.078278,56.054363
43,2021-08-02,None,2062.822998,NaN,1909.767090,95.704643,57.351151


In [19]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(X_tst['y'],forecast['yhat1'])
print("MAPE",round(mape,4))

MAPE 37.0565


In [20]:
from sklearn.metrics import mean_squared_error

mean_squared_error(X_tst['y'],forecast['yhat1'])

1450936.1325055864

Prediction for next 30 days

In [21]:
future = m.make_future_dataframe(df,  periods=30)
forecast = m.predict(future)
forecast

,ds,y,yhat1,residual1,trend,season_yearly,season_weekly
0,2021-08-03,None,2067.412598,NaN,1909.606934,97.611237,60.194504
1,2021-08-04,None,2068.374756,NaN,1909.446533,99.508659,59.419403
2,2021-08-05,None,2071.900879,NaN,1909.286255,101.381348,61.233120
3,2021-08-06,None,2068.393555,NaN,1909.125977,103.213303,56.054363
4,2021-08-09,None,2074.300049,NaN,1908.645264,108.303619,57.351151
5,2021-08-10,None,2078.491211,NaN,1908.484863,109.811897,60.194504
6,2021-08-11,None,2078.944092,NaN,1908.324707,111.200005,59.419403
7,2021-08-12,None,2081.850830,NaN,1908.164429,112.453194,61.233120
8,2021-08-13,None,2077.615723,NaN,1908.004150,113.557411,56.054363
9,2021-08-16,None,2080.723633,NaN,1907.523315,115.849243,57.351151


Train Test split

In [22]:
cut_date = '2021-07-19'
mask1 = (df['ds'] <= cut_date)
mask2 = (df['ds'] > cut_date)

X_tr = df.loc[mask1]
X_tst = df.loc[mask2]
print("train shape",X_tr.shape)
print("test shape",X_tst.shape)

train shape (1232, 2)
test shape (9, 2)


In [34]:
m = NeuralProphet(learning_rate=0.0001, seasonality_mode='multiplicative', num_hidden_layers=20, batch_size=4)
#metrics = m.fit(df, validate_each_epoch=True, valid_p=0.2)
model = m.fit(X_tr, validate_each_epoch=True, freq="B", epochs=1000) # epoch=1000


INFO: nprophet.utils - set_auto_seasonalities: Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
Epoch[1000/1000]: 100%|██████████| 1000/1000 [07:27<00:00,  2.23it/s, SmoothL1Loss=0.00158, MAE=62.1, RegLoss=0, MAE_val=875, SmoothL1Loss_val=0.231]


In [32]:
future = m.make_future_dataframe(X_tr, periods=9)
forecast = m.predict(future)
forecast.head()

,ds,y,yhat1,residual1,trend,season_yearly,season_weekly
0,2021-07-20,None,2062.780273,NaN,2480.181152,0.059817,-0.325472
1,2021-07-21,None,2061.482422,NaN,2479.890625,0.059834,-0.326179
2,2021-07-22,None,2062.920410,NaN,2479.599854,0.059935,-0.325229
3,2021-07-23,None,2060.486816,NaN,2479.309326,0.060126,-0.326833
4,2021-07-26,None,2065.877441,NaN,2478.437256,0.061290,-0.324155


In [33]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(X_tst['y'],forecast['yhat1'])
print("MAPE",round(mape,4))

MAPE 35.4935


In [26]:
from sklearn.metrics import mean_squared_error

mean_squared_error(X_tst['y'],forecast['yhat1'])

1263334.6206335293

Prediction for next 30 days

In [27]:
future = m.make_future_dataframe(df,  periods=30)
forecast = m.predict(future)
forecast

,ds,y,yhat1,residual1,trend,season_yearly,season_weekly
0,2021-08-03,None,2085.557617,NaN,1919.029541,109.959061,56.568985
1,2021-08-04,None,2086.394287,NaN,1918.862671,111.239738,56.291870
2,2021-08-05,None,2085.456543,NaN,1918.695801,112.490883,54.269917
3,2021-08-06,None,2088.139160,NaN,1918.528931,113.701454,55.908691
4,2021-08-09,None,2094.458008,NaN,1918.028198,116.975922,59.454014
5,2021-08-10,None,2092.340088,NaN,1917.861328,117.909798,56.568985
6,2021-08-11,None,2092.731934,NaN,1917.694336,118.745689,56.291870
7,2021-08-12,None,2091.270020,NaN,1917.527588,119.472412,54.269917
8,2021-08-13,None,2093.348633,NaN,1917.360596,120.079216,55.908691
9,2021-08-16,None,2097.394043,NaN,1916.859985,121.079964,59.454014


In [39]:
model = NeuralProphet(daily_seasonality=False)
metrics = model.fit(X_tr, freq="D")

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch[51/51]: 100%|██████████| 51/51 [00:04<00:00, 12.49it/s, SmoothL1Loss=0.00177, MAE=115, RegLoss=0]


In [40]:
future = m.make_future_dataframe(X_tr, periods=9)
forecast = m.predict(future)
forecast.head()

,ds,y,yhat1,residual1,trend,season_yearly,season_weekly
0,2021-07-20,None,2074.094238,NaN,1552.257812,0.146648,0.664549
1,2021-07-21,None,2072.121582,NaN,1552.158447,0.146140,0.662269
2,2021-07-22,None,2074.531006,NaN,1552.059204,0.145812,0.666623
3,2021-07-23,None,2070.343750,NaN,1551.959839,0.145680,0.660523
4,2021-07-26,None,2073.737793,NaN,1551.661621,0.146606,0.665716


In [41]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(X_tst['y'],forecast['yhat1'])
print("MAPE",round(mape,4))

MAPE 35.2016
